In [81]:
import re
import functools

# input_file = 'input_example.txt'
input_file = 'input_full.txt'

with open(input_file) as file:
    lines = file.readlines()
lines = [[int(i) for i in re.findall(r'-?\d+', line)] for line in lines]

blueprints = {}
for line in lines:
    blueprints[line[0]] = {
        'ore_robot_ore_cost': line[1],
        'clay_robot_ore_cost': line[2],
        'obsidian_robot_ore_cost': line[3],
        'obsidian_robot_clay_cost': line[4],
        'geode_robot_ore_cost': line[5],
        'geode_robot_obsidian_cost': line[6],
    }

In [82]:
@functools.lru_cache(maxsize=None)
def solve(maxtime, time, r0, r1, r2, r3, m0, m1, m2, m3, r0_m0, r1_m0, r2_m0, r2_m1, r3_m0, r3_m2):
    # r0: ore robots, r1: clay robots, r2: obsidian robots, r3: geode robots
    # m0: ores, m1: clays, m2: obsidians, m3: geodes
    # r0_m0, r1_m0, r2_m0, r2_m1, r3_m0, r3_m2: blueprint
    b = [r0_m0, r1_m0, r2_m0, r2_m1, r3_m0, r3_m2]

    if time == maxtime+1:
        return m3

    max_geodes = 0

    if m0 < max(r0_m0, r1_m0, r2_m0, r3_m0): # buy nothing
        max_geodes = max(max_geodes, solve(maxtime, time+1, r0, r1, r2, r3, m0+r0, m1+r1, m2+r2, m3+r3, *b))
    if m0 >= r0_m0 and r0 < max(r0_m0, r1_m0, r2_m0, r3_m0): # buy ore robot
        max_geodes = max(max_geodes, solve(maxtime, time+1, r0+1, r1, r2, r3, m0+r0-(r0_m0), m1+r1, m2+r2, m3+r3, *b))
    if m0 >= r1_m0 and r1 < r2_m1: # buy clay robot
        max_geodes = max(max_geodes, solve(maxtime, time+1, r0, r1+1, r2, r3, m0+r0-(r1_m0), m1+r1, m2+r2, m3+r3, *b))
    if m0 >= r2_m0 and m1 >= r2_m1 and r2 < r3_m2: # buy obsidian robot
        max_geodes = max(max_geodes, solve(maxtime, time+1, r0, r1, r2+1, r3, m0+r0-(r2_m0), m1+r1-(r2_m1), m2+r2, m3+r3, *b))
    if m0 >= r3_m0 and m2 >= r3_m2: # buy geode robot
        max_geodes = max(max_geodes, solve(maxtime, time+1, r0, r1, r2, r3+1, m0+r0-r3_m0, m1+r1, m2+r2-r3_m2, m3+r3, *b))

    return max_geodes

In [83]:
result = 0
for i in range(1,len(blueprints)+1):
    result += i * solve(24, 1, *[1,0,0,0], *[0,0,0,0], *blueprints[i].values())
print(f'Part 1: {result}')

Part 1: 1834


In [84]:
result = 1
for i in range(1,4):
    result *= solve(32, 1, *[1,0,0,0], *[0,0,0,0], *blueprints[i].values())
print(f'Part 2: {result}')

Part 2: 2240
